In [2]:
# adding maxspeed
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp

In [75]:
# Creating the test graph
G = nx.MultiDiGraph()
# G.add_edge(1, 2, weight=1, isNew=False)
G.add_edge(1, 2, weight=np.nan, noWay=0, isClosed=0, length=20, speed=18, maxSpeed=40)
G.add_edge(1, 3, weight=np.nan, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20)
G.add_edge(2, 4, weight=np.nan, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=40)
G.add_edge(3, 4, weight=np.nan, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=10)
G.add_edge(4, 5, weight=np.nan, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20)
desiredPath = [1, 3, 4]

In [79]:
# # Other  direction, for testing
#G.add_edge(2, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=18, maxSpeed=20)
# G.add_edge(3, 1, weight=np.nan, noWay=0, isClosed=0, length=20, speed=15, maxSpeed=20)
# G.add_edge(4, 2, weight=np.nan, noWay=0, isClosed=0, length=20, speed=10, maxSpeed=20)
# G.add_edge(4, 3, weight=np.nan, noWay=0, isClosed=0, length=20, speed=7, maxSpeed=20)
# G.add_edge(5, 4, weight=np.nan, noWay=0, isClosed=1, length=20, speed=5, maxSpeed=20)

In [80]:
# Helper Functions

def calculateWeight(data):
    inf = 1e6

    #weight = getInverse(data['speed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
    
    weight = getInverse(data['maxSpeed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
        
    return weight

def updateEdgeWeight(graph, source, target, data):
    graph[source][target][0]['weight'] = calculateWeight(data)

def updateGraphWeights(graph):
    for (i, j, data) in graph.edges(data=True):
        updateEdgeWeight(graph, i, j, data)
    
def getPathWeight(path, graph):
    weights = 0
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        weights += graph[source][target][0]['weight']
        
    return weights
        
def getInverse(speed):
    return 1/speed

def addReverseEdges(graph):
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            graph.add_edge(
                            j, i, 
                            weight = np.nan,
                            noWay = 1,
                            isClosed = 0,
                            length = 0,
                            speed = graph[i][j][0]['speed'],
                            maxSpeed = graph[i][j][0]['maxSpeed']
                        )
            updateEdgeWeight(graph, j, i, graph[j][i][0])
    
def cleanGraphAttributes(graph):
    for (i, j) in graph.edges():
        if graph[i][j][0]['isClosed'] == 1:
            graph[i][j][0]['length'] = 0
    

In [81]:
cleanGraphAttributes(G)
updateGraphWeights(G)

In [86]:
#TODO: add the switch between speed and max speed

def inverseShortestPath(original_graph, desiredPath):
    # Constants
    inf = 1e6
    epsilon = 1e-15
    possibleMaxSpeeds = [10, 20, 30, 40, 50]
    inversePossibleMaxSpeeds = [getInverse(s) for s in possibleMaxSpeeds]
    inversePossibleMaxSpeeds = np.asarray(inversePossibleMaxSpeeds)
    
    # adding reverse edges.
    graph = original_graph.copy()
    addReverseEdges(graph)
    
    # Some graph and path data
    n = len(graph.nodes())
    m = len(graph.edges())
    source = desiredPath[0]
    target = desiredPath[len(desiredPath) - 1]

    # Original Graph data
    noWay = []
    areClosed = []
    inverseSpeeds = []
    inverseMaxSpeeds = []
    lengths = []
    maxSpeeds_H1E = []
    
    # Edges data, and their indecies
    edges = []
    edgeIndex = {}
    for (i, j, data) in graph.edges(data=True):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Add edge
        edges.append([i, j])
        
        # Data:
        noWay.append(data['noWay'])
        areClosed.append(data['isClosed'])
        inverseSpeeds.append(getInverse(data['speed']))
        inverseMaxSpeeds.append(getInverse(data['maxSpeed']))
        lengths.append(data['length'])
        
        # hot 1 encoding original data
        hot1E = []
        for ms in possibleMaxSpeeds:
            if data['maxSpeed'] == ms:
                hot1E.append(1)
            else:
                hot1E.append(0)
        maxSpeeds_H1E.append(hot1E)
        
        
    # Nodes data, and their indecies
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)
        
        
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            if (neighbour, nodes[i]) in edgeIndex:
                j = edgeIndex[neighbour, nodes[i]]
                A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1
            
            
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edgeIndex[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edgeIndex[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
        


    # LP (ISP):
    
    # Original Data as numpy array. (for mathematical applications)
    noWay_original = np.asarray(noWay)
    areClosed_original = np.asarray(areClosed)
    inverseSpeeds_original = np.asarray(inverseSpeeds)
    inverseMaxSpeeds_original = np.asarray(inverseMaxSpeeds)
    maxSpeeds_H1E_original = np.asarray(maxSpeeds_H1E)
    
    
    # Variables
    pi_ = cp.Variable(len(nodes)) #(2d)
    lambda_ = cp.Variable(len(edges)) #(2d)
    
    noWay_ = cp.Variable(len(noWay_original), boolean=True)
    areClosed_ = cp.Variable(len(areClosed_original), boolean=True)
    #inverseSpeeds_ = cp.Variable(len(inverseSpeeds_original))
    inverseMaxSpeeds_ = cp.Variable(len(inverseMaxSpeeds_original))
    maxSpeeds_H1E_ = cp.Variable(maxSpeeds_H1E_original.shape, boolean=True)
        
        
    # Constraints
    constraints = []
    
    # (2b) & (2c) 
    for j in range(len(edges)):
        #d_j = inverseSpeeds_[j] * lengths[j]  + inf * noWay_[j] + inf * areClosed_[j]
        d_j = inverseMaxSpeeds_[j] * lengths[j]  + inf * noWay_[j] + inf * areClosed_[j]
        
        if xzero[j] == 1:
        # sum_i a_ij * pi_i = d_j,              for all j in desired path (2b)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
        else:
        # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path (2c)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
        
        
    #(2e)
    for j in range(len(edges)):
        if xzero[j] == 0:
            constraints.append( lambda_[j] >= 0 )
            
            
    # Hot 1 Encoding    
    for row in maxSpeeds_H1E_:
        constraints.append( sum(row) == 1)
            
    
    for j in range(len(edges)):
        if xzero[j] == 0:
            #if not in desired path, do not change the data 
            constraints.append( noWay_[j] == noWay_original[j] )
            constraints.append( areClosed_[j] == areClosed_original[j] )
            #constraints.append( inverseSpeeds_[j] == inverseSpeeds_original[j] )
            constraints.append( inverseMaxSpeeds_[j] == inverseMaxSpeeds_original[j] )
            
            constraints.append( maxSpeeds_H1E_[j] == maxSpeeds_H1E_original[j] )
        
        
        # # inverseSpeed is at least the max speed; ie speed is at most max speed.
        # else:
        #     constraints.append(  inverseSpeeds_[j] >= inverseMaxSpeeds_original[j] )
        
        # Hot 1 Encoding
        constraints.append( inverseMaxSpeeds_[j] == inversePossibleMaxSpeeds.T @ maxSpeeds_H1E_[j] )

        
    
    
    
    # Cost function, split up
    #cost1 = cp.norm1(inverseSpeeds_ - inverseSpeeds_original)
    cost1 = cp.norm1(inverseMaxSpeeds_ - inverseMaxSpeeds_original)
    cost2 = cp.norm1(noWay_ - noWay_original)
    cost3 = cp.norm1(areClosed_ - areClosed_original)
            
            
    # Final Cost funnction
    cost = cost1 + cost2 + cost3
    
        
    # Forming the problem
    prob = cp.Problem(cp.Minimize(cost), constraints)
    
    # Solve the problem
    #prob.solve(solver=cp.GUROBI, verbose=True) #Detailed
    prob.solve(solver=cp.GUROBI) # using gurobi
    print("\nThe optimal value is", prob.value)
    
    
    # print('original speedInv: ', inverseSpeeds_original)
    # print('optimal speedInv: ', inverseSpeeds_.value)
    # print('\n')
    print('original speedMaxInv: ', inverseMaxSpeeds_original)
    print('optimal speedMaxInv: ', inverseMaxSpeeds_.value)
    print('\n')
    print('original noWay: ', noWay_original)
    print('optimal noWay: ', noWay_.value)
    print('\n')
    print('original areClosed: ', areClosed_original)
    print('optimal areClosed: ', areClosed_.value)
    
    
    newGraph = nx.MultiDiGraph()
    
    for (i, j), index in edgeIndex.items():
        newGraph.add_edge(
                            i, j, 
                            #weight = w_.value[index], 
                            weight = np.nan, 
                            noWay = noWay_.value[index], 
                            isClosed = areClosed_.value[index], 
                            length=graph[i][j][0]['length'], 
                            # speed = getInverse(inverseSpeeds_.value[index]), 
                            # maxSpeed=graph[i][j][0]['maxSpeed']
                            speed = graph[i][j][0]['speed'],
                            maxSpeed = getInverse(inverseMaxSpeeds_.value[index])
                        )
        
    updateGraphWeights(newGraph)
        
        
    sp = nx.shortest_path(newGraph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    
    desiredPathWeight = getPathWeight(desiredPath, newGraph)
    optimalPathWeight = getPathWeight(sp, newGraph)
    print('\n')
    
    if (desiredPathWeight - optimalPathWeight) <= epsilon:
        print('The desired Path is equal to the Shortest Path')
    elif desiredPathWeight < optimalPathWeight:
        print('The desired Path is better than the Shortest Path')
    elif desiredPathWeight > optimalPathWeight:
        print('The desired Path is worse than the Shortest Path')
    
    
    print('Optimal Path Weight = ', optimalPathWeight)
    print('The path is: ', sp)
    print('numbers after decimal point: ', len(str(optimalPathWeight).replace('.','')) - 1)
    print('\n')
    print('Desired Path Weight = ', desiredPathWeight)
    print('The path is: ', desiredPath)
    print('numbers after decimal point: ', len(str(desiredPathWeight).replace('.','')) - 1)
        
    return newGraph


In [87]:
# Solving using the function

new_graph = inverseShortestPath(G, desiredPath)


The optimal value is 0.1
original speedMaxInv:  [0.025 0.05  0.025 0.025 0.1   0.05  0.05  0.025 0.1   0.05 ]
optimal speedMaxInv:  [0.025 0.025 0.025 0.025 0.025 0.05  0.05  0.025 0.1   0.05 ]


original noWay:  [0 0 0 1 0 1 0 1 1 1]
optimal noWay:  [-0.  0. -0.  1.  0.  1. -0.  1.  1.  1.]


original areClosed:  [0 0 0 0 0 0 1 0 0 0]
optimal areClosed:  [-0.  0. -0. -0.  0. -0.  1. -0. -0. -0.]


The desired Path is equal to the Shortest Path
Optimal Path Weight =  1.0
The path is:  [1, 2, 4]
numbers after decimal point:  1


Desired Path Weight =  1.0
The path is:  [1, 3, 4]
numbers after decimal point:  1


In [88]:
G.edges(data='maxSpeed')
#G.edges(data='weight')
#G.edges(data=True)

OutMultiEdgeDataView([(1, 2, 40), (1, 3, 20), (2, 4, 40), (3, 4, 10), (4, 5, 20)])

In [89]:
new_graph.edges(data='maxSpeed')
#new_graph.edges(data='weight')
#new_graph.edges(data=True)

OutMultiEdgeDataView([(1, 2, 40.0), (1, 3, 40.0), (2, 4, 40.0), (2, 1, 40.0), (3, 4, 40.0), (3, 1, 20.0), (4, 5, 20.0), (4, 2, 40.0), (4, 3, 10.0), (5, 4, 20.0)])

In [ ]:
    #TODO:
    # weights for L1norm (not-on-path-penalty) # Why do we use this???????
    # penalty = np.array([1]*len(w_original))
    # for i in range(len(nodes)):
    #     if nodes[i] not in desiredPath[1:-1]:
    #         penalty[i] = 10
    
    #Check why the penalty??
    #cost = cp.norm1(cp.multiply(w_ - w_original, penalty))

In [10]:
#ORIGINAL
# import similaritymeasures
# import matplotlib
# import matplotlib.pyplot as plt
# import time
# import random
# import pdb
# import os
# import rospy
# from dynamic_reconfigure.server import Server
# from recast_explanations_ros.cfg import ExplanationsConfig
# from geometry_msgs.msg import Point
# from visualization_msgs.msg import Marker, MarkerArray
# from std_msgs.msg import ColorRGBA
# from recast_ros.srv import RecastProjectSrv, RecastProjectSrvRequest
# from recast_ros.srv import RecastPathSrv, RecastPathSrvRequest
# from recast_ros.msg import RecastGraph, RecastGraphNode
# from itertools import islice, product
# from tabulate import tabulate
# from queue import PriorityQueue


def optInvMILP2(graph, desiredPath, areaCosts, allowedAreaTypes, exact=True, verbose=True):

  # variables:
  #   x_j:      indicator variable, whether edge j is part of the shortest path
  #   A_ij:     node-arc incidence matrix (rows are nodes, columns are edges) = 1 if j leaves i, -1 if j enters i, 0 otherwise
  #   b_i:      difference between entering and leaving edges = 1 if i start, -1 if i target, 0 otherwise
  #   pi_i:     dual variable
  #   lambda_j: dual variable
  #   c_j:      edge cost = dist_j * ac_0 * l_0 + dist_j * ac_1 * l_1 + ... = sum_(k in areas) dist_j * ac_k * l_ik
  #   l_ik:     node area one-hot encoding

  # problems:
  #   SP:  min  c.x,
  #        s.t. Ax=b, x>=0
  #   ISP: min  |l-l'|,
  #        s.t. sum_i a_ij * pi_i = sum_(k in areas) dist_j * ac_k * l_ik,              for all j in desired path
  #             sum_i a_ij * pi_i + lambda_j = sum_(k in areas) dist_j * ac_k * l_ik,   for all j not in desired path
  #             sum_k l_ik = 1                                                          for all i
  #             lambda >= 0,                                                            for all j not in desired path.

  cost_type = "label_changes"   # "weights" or "label_changes"

  # auxiliary variables
  edge2index = {}
  edges = []
  edge2varnodeindex = {}
  varnodes = []
  weights = []
  for (i,j) in graph.edges:
    edge2index[i,j] = len(edges)
    edges.append([i,j])
    edge2index[j,i] = len(edges)
    edges.append([j,i])
    if not graph.nodes[i]["portal"]:
      vn = i
    else:
      vn = j
    if vn in varnodes:
      idx = varnodes.index(vn)
    else:
      idx = len(varnodes)
      varnodes.append(vn)
    edge2varnodeindex[i,j] = idx
    edge2varnodeindex[j,i] = idx
    weights.append(graph[i][j]["weight"])
  node2index = {}
  nodes = []
  for n in graph.nodes:
    node2index[n] = len(nodes)
    nodes.append(n)
    if n == desiredPath[0]:
      s = node2index[n]
    if n == desiredPath[-1]:
      t = node2index[n]
  weights = np.array(weights)

  # l_original
  l_original = np.zeros(len(varnodes) * len(allowedAreaTypes))
  for (i,j) in graph.edges:
    idx = edge2varnodeindex[i,j]
    node = varnodes[idx]
    for k in range(len(allowedAreaTypes)):
      if allowedAreaTypes[k] == graph.nodes[node]["area"]:
        l_original[len(allowedAreaTypes) * idx + k] = 1
      else:
        l_original[len(allowedAreaTypes) * idx + k] = 0

  # Ax = b
  A = np.zeros([len(nodes), len(edges)])
  b = np.zeros(len(nodes))
  for i in range(len(nodes)):
    for nei in graph.adj[nodes[i]]:
      j = edge2index[nodes[i], nei]
      A[i,j] = 1
      j = edge2index[nei, nodes[i]]
      A[i,j] =-1
    if i == s:
      b[i] = 1
    if i == t:
      b[i] =-1

  # optimal x
  path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
  xstar = np.zeros(len(edges))
  for p in range(len(path)-1):
    j = edge2index[path[p], path[p+1]]
    xstar[j] = 1

  # desired x
  xzero = np.zeros(len(edges))
  for p in range(len(desiredPath)-1):
    j = edge2index[desiredPath[p], desiredPath[p+1]]
    xzero[j] = 1

  # weights for L1norm (not-on-path-penalty)
  w = np.array([1]*len(l_original))
  for i in range(len(varnodes)):
    if varnodes[i] not in desiredPath[1:-1]:
      for k in range(len(allowedAreaTypes)):
        w[len(allowedAreaTypes) * i + k] *= 10

  # inverse optimization problem
  l_ = cp.Variable(len(l_original), boolean=True)
  pi_ = cp.Variable(len(nodes))
  lambda_ = cp.Variable(len(edges))
  # cost
  if cost_type == "weights":
    cost = 0
    j = 0
    for edge in graph.edges:
      i = edge2varnodeindex[edge[0], edge[1]]
      # edge's new cost d_j = sum_(k in areas) dist_j * ac_k * l_ik
      d_j = 0
      dist_j = dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
      for k in range(len(allowedAreaTypes)):
        ac_k = areaCosts[allowedAreaTypes[k]]
        d_j += dist_j * ac_k * l_[len(allowedAreaTypes) * i + k]
      cost += cp.abs(d_j - weights[j])
      j += 1
  else:
    cost = cp.norm1(cp.multiply(l_ - l_original, w))  # cost = cp.norm1(l_ - l_original)
  # constraints
  constraints = []
  for j in range(len(edges)):
    edge = edges[j]
    i = edge2varnodeindex[edge[0], edge[1]]
    # edge's new cost d_j = sum_(k in areas) dist_j * ac_k * l_ik
    d_j = 0
    dist_j = dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
    for k in range(len(allowedAreaTypes)):
      ac_k = areaCosts[allowedAreaTypes[k]]
      d_j += dist_j * ac_k * l_[len(allowedAreaTypes) * i + k]
    if xzero[j] == 1:
      # sum_i a_ij * pi_i = d_j,              for all j in desired path
      constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
    else:
      # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path
      constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
  # sum_k l_ik = 1, for all i
  for i in range(len(varnodes)):
    idx = len(allowedAreaTypes) * i
    constraints.append( cp.sum(l_[idx:idx+len(allowedAreaTypes)]) == 1 )
  # lambda >= 0, for all j not in desired path.
  for j in range(len(edges)):
    if xzero[j] == 0:
      constraints.append( lambda_[j] >= 0 )

  # solve with cvxpy
  prob = cp.Problem(cp.Minimize(cost), constraints)
  if exact:
    value = prob.solve(solver=cp.MOSEK, mosek_params={"MSK_DPAR_MIO_MAX_TIME":-1, "MSK_DPAR_MIO_TOL_REL_GAP":0, "MSK_IPAR_MIO_CUT_CLIQUE":0, "MSK_IPAR_MIO_CUT_CMIR":0, "MSK_IPAR_MIO_CUT_GMI":0, "MSK_IPAR_MIO_CUT_SELECTION_LEVEL":0, "MSK_IPAR_MIO_FEASPUMP_LEVEL":1, "MSK_IPAR_MIO_VB_DETECTION_LEVEL":1}, verbose=verbose)
  else:
    value = prob.solve(solver=cp.GUROBI, verbose=verbose)
  if value == float('inf'):
    rospy.loginfo("  inverse shortest path MILP failed")
    return []

  # TODO: can solve every 90s, warm starting from previous solution, until optimality or time budget
  #       this way can return multiple solutions of better and better cost

  # new graph
  newGraph = graph.copy()
  changed = 0
  for j in range(len(edges)):
    edge = edges[j]
    i = edge2varnodeindex[edge[0], edge[1]]
    area = -1
    for k in range(len(allowedAreaTypes)):
      if l_.value[len(allowedAreaTypes) * i + k] == 1:
        area = allowedAreaTypes[k]
        break
    if area != graph[edge[0]][edge[1]]["area"]:
      changed += 1
    newGraph[edge[0]][edge[1]]["area"] = area
    newGraph[edge[0]][edge[1]]["cost"] = areaCosts[area]
    newGraph[edge[0]][edge[1]]["weight"] = areaCosts[area] * dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
    if not newGraph.nodes[edge[0]]["portal"]:
      newGraph.nodes[edge[0]]["area"] = area
      newGraph.nodes[edge[0]]["cost"] = areaCosts[area]
    if not newGraph.nodes[edge[1]]["portal"]:
      newGraph.nodes[edge[1]]["area"] = area
      newGraph.nodes[edge[1]]["cost"] = areaCosts[area]

  # sanity check
  new_path = nx.shortest_path(newGraph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
  if getCost(graph, new_path) != getCost(graph, desiredPath):
    rospy.logwarn("  new shortest path is not the desired one")
  elif verbose:
    rospy.loginfo("  inverse shortest path: success")

  # changed entries
  if verbose:
    rospy.loginfo("  changed labels: %d" % changed)

  #pdb.set_trace()
  return l_.value, newGraph
